In [9]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
from imp import reload
import nice_layers as vnn
reload(nice_layers)

<module 'nice_layers' from '/Users/davidclark/Projects/VectorizedNets/nice_layers.py'>

In [5]:
model = nn.Sequential(V.VecLinear(10, 150, 200),
                      V.VecLinear(10, 200, 50))

In [7]:
input = torch.randn(256, 10, 150)
output = model(input)
output.shape

torch.Size([256, 10, 50])

In [8]:
model

Sequential(
  (0): VecLinear()
  (1): VecLinear()
)

In [191]:
reload(vnn)

model = nn.Sequential(vnn.Conv2d(10, 30, 6, 5),
                      vnn.tReLU(),
                      vnn.AvgPool2d(2),
                      vnn.Conv2d(10, 6, 16, 5),
                      vnn.tReLU(),
                      vnn.AvgPool2d(2),
                      vnn.Conv2d(10, 16, 120, 5),
                      vnn.tReLU(),
                      vnn.Flatten(),
                      vnn.Linear(10, 120, 84),
                      vnn.tReLU(),
                      vnn.Dropout(),
                      vnn.Linear(10, 84, 1))

In [192]:
input = torch.randn(1, 10, 30, 32, 32)


In [193]:
output = model(input)
output.shape

Instantiated t with shape (10, 6, 28, 28)
Instantiated t with shape (10, 16, 10, 10)
Instantiated t with shape (10, 120, 1, 1)
Instantiated t with shape (10, 84)


torch.Size([1, 10, 1])

In [187]:
for module in model: module.post_step_callback()


In [120]:
do = nn.Dropout(p=0.5)
x = torch.ones(5, 3)
do(x)

tensor([[2., 0., 0.],
        [2., 2., 2.],
        [0., 0., 0.],
        [0., 0., 0.],
        [2., 0., 2.]])

In [166]:
class Dropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p
    def forward(self, input):
        if self.training:
            mask_shape = (input.shape[0],) + input.shape[2:]
            mask = (torch.rand(mask_shape, device=input.device) > self.p).float()
            output = input * mask[:, None] * (1. / (1. - self.p))
            return output
        else:
            return input

In [175]:
do = Dropout(p=0.9)
x = torch.ones(3, 10, 5)
do.train()
do(x)

tensor([[[ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0., 10.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,

In [168]:
do.

tensor([[[0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.],
         [0., 2., 2., 2., 2.]],

        [[0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.],
         [0., 2., 0., 0., 0.]],

        [[0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.],
         [0., 0., 2., 2., 2.]]])